# Introduction

In this tutorial, we will explore a basic example of forecasting multiple time series and go over the key components of the forecasting pipeline provided by the `Tsururu` library.

Let's import everything we will need.

In [1]:
import warnings

warnings.filterwarnings("ignore")

from typing import List, Optional, Union

import numpy as np
import pandas as pd
import time
import polars as pl

from tsururu.dataset import Pipeline, TSDataset, TSDatasetPolars, TSDatasetNumba, TSDatasetNumbaPolars
from tsururu.model_training.trainer import MLTrainer
from tsururu.model_training.validator import KFoldCrossValidator
from tsururu.models.boost import CatBoost
from tsururu.strategies import RecursiveStrategy


In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

There are several main objects to look out for when working with the library:
1) `TSDataset`.
2) `Pipeline` and `Transformers`
3) `Strategy`.
4) `Model`.

# Working with Data

### TSDataset

This class is needed to store data and meta-information about it.

To initialise it is necessary to submit the data in `pd.DataFrame` format and define some meta-information about roles that necessary for solving the task of time series forecasting: `id`, `date`, `target`.

In [3]:
df_path = "datasets/global/simulated_data_to_check.csv"

dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [4]:
start = time.time()
dataset = TSDataset(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)
end = time.time()
print(end - start)

freq: Day; period: 1
0.02325606346130371


In [5]:
start = time.time()
dataset = TSDatasetPolars(
    data=pl.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)
end = time.time()
print(end - start)

freq: Day; period: 1
0.048441171646118164


In [6]:
start = time.time()
dataset = TSDatasetNumba(
    data=pd.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)
end = time.time()
print(end - start)

freq: Day; period: 1
0.02024221420288086


In [7]:
start = time.time()
dataset = TSDatasetNumbaPolars(
    data=pl.read_csv(df_path),
    columns_params=dataset_params,
    print_freq_period_info=True,
)
end = time.time()
print(end - start)

freq: Day; period: 1
0.01135706901550293


### Pipeline and Transformers

The Pipeline class is designed to create and apply a sequence of transformations (transformers) to time series data. It is used for data preprocessing, feature and target generation, as well as performing transformations required for forecasting models.

In this tutorial, we will cover a simplified approach to initializing the Pipeline. For detailed information on the available transformers and methods for building a pipeline, refer to Tutorial 3 (Tutorial_3_Pipeline.ipynb).

#### What kind of transformers are there?

Special attention should be paid to the `Transformer` class: the elements of the pipeline that are responsible for transforming the values of a series and generating features. `Pipeline` class is a wrapper over transformers which is needed to provide some additional methods and functions above them.

There are two types of transformers that are used to collect pipelines:
- `Union` transformers;
- `Sequential` transformers.

Below is a list of available Transformers: 
- **StandardScalerTransformer** *(Series2Series)*.
- **DifferenceNormalizer** *(Series2Series)*: subtract the previous value or divide by it.
- **TimeToNumGenerator** and **DateSeasonsGenerator** *(Series2Series)* - generator for seasonal features by dates.
- **LabelEncodingTransformer** and **OneHotEncodingTransformer** *(Series2Series)* - encoders for categorical features.
- **MissingValuesImputer** *(Series2Series)*.
- **LagTransformer** *(Series2Features)* - generator for lags. 
- **LastKnownNormalizer** *(Features2Features)*: normalize all lags by the last known one: divide by it or subtract.

!!!The lag transformer must necessarily be present in the sequential transformer, otherwise the features will not be generated.!!!

Finally, to generate targets, you need to use **TargetGenerator**.

#### Transformers must be assembled in order!

The __SeriesToSeries__ transformers should come first, followed by the LagTransformer and TargetGenerator (__SeriesToFeatures__), and then the __FeaturesToFeatures__ transformers.

!!!Thus, **StandardScalerNormalizer** and **DifferenceNormalizer** should be before **LagTransformer** and **LastKnownNormalizer** after it!!!

#### How to build a Pipeline?

In [8]:
pipeline_easy_params = {
    "target_lags": 3,
    "date_lags": 1,
#    "exog_lags": 1,  # Uncomment this line if you have exogenous features
    # One from ["none", "standard_scaler", "difference_normalizer", "last_known_normalizer"]
    "target_normalizer": "standard_scaler",
    # One from ["none", "delta", "ratio"]  (MUST BE "none" for "standard_scaler" and NOT "none" for others)
    "target_normalizer_regime": "none",
}

In [9]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

#### Can I use exogenous variables in the pipeline?

Yes! Exogenous variables can also be specified here. Just include them in your pipeline.

However, their operation is currently tested only for the `MIMOStrategy` in global-modelling. For other strategies support of additional variables is under development.

In [10]:
# pipeline_params["exog_group_1"] = {
#     "columns": ["value"],
#     "features": {
#         "StandardScalerTransformer":
#             {
#                 "transform_target": False, 
#                 "transform_features": True
#             },
#         "LagTransformer": {"lags": 7},
#     },
# }

__Make sure you have the transform_target = False flag for exogenous features!__

#### Model, Validator and Trainer

- `Model`:
  - The model is separate from the strategy. Any model can be run in any strategy if it supports this input and output format.
  - You can use on of the implemented ML models (for instance, GBM (Gradient Boosting Machine)).
- `Validator`:
  - The validator is responsible for setting up the validation process, which includes creating training and validation folds. It ensures that the data is split correctly so that the model’s performance can be accurately assessed. 
- `Trainer`:
  - The trainer is the component that trains the model with provided validator. 
  - It is necessary to choose a trainer in accordance with the type of model (ML, DL, stats).

In [11]:
# Configure the model parameters
model = CatBoost
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation = KFoldCrossValidator
validation_params = {
    "n_splits": 2,
}

trainer_params = {}

trainer = MLTrainer(
    model,
    model_params,
    validation,
    validation_params,
)

#### Strategy

- _Recursive:_ 
    - one model for all points of the forecast horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: a prediction is iteratively made one point ahead, and then this prediction is used to further shape the features in the test data. 
- _Recursive-reduced:_
    - one model for all points in the prediction horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: features are generated for all test observations at once, unavailable values are replaced by NaN.
- _Direct:_ 
    - individual models for each point in the prediction horizon. 
- _MultiOutput (MIMO - Multi-input-multi-output):_
    - one model that learns to predict the entire prediction horizon. 
    - __Also, this strategy supports the presence of `exogenous features` (only for local- or global-modelling).__
- _FlatWideMIMO:_.
    - mixture of Direct and MIMO, fit one model, but uses deployed over horizon Direct's features.
    - __Number of `lags for datetime features` should be equal to `horizon` while using this strategy.__

In [12]:
horizon = 3
history = 7

In [13]:
strategy = RecursiveStrategy(horizon, history, trainer, pipeline)

In [14]:
fit_time, _ = strategy.fit(dataset)

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


AttributeError: 'TimedeltaIndex' object has no attribute 'iloc'

In [12]:
forecast_time, current_pred = strategy.predict(dataset)

In [13]:
current_pred

,id,date,value
0,0,2022-09-27,1994.783481
1,0,2022-09-28,1994.440142
2,0,2022-09-29,1993.71401
3,1,2022-09-27,2994.783481
4,1,2022-09-28,2994.440142
5,1,2022-09-29,2993.71401
6,2,2022-09-27,3994.783481
7,2,2022-09-28,3994.440142
8,2,2022-09-29,3993.71401
9,3,2022-09-27,4994.783481


## Backtest validation of pipeline

In [14]:
ids, test, pred, fit_time, forecast_time = strategy.back_test(dataset, cv=1)

0:	learn: 0.9617997	test: 0.9655311	best: 0.9655311 (0)	total: 978us	remaining: 977ms
500:	learn: 0.0039542	test: 0.0041461	best: 0.0041461 (500)	total: 390ms	remaining: 388ms
999:	learn: 0.0024434	test: 0.0026360	best: 0.0026360 (999)	total: 844ms	remaining: 0us

bestTest = 0.002636039164
bestIteration = 999

Fold 0. Score: 0.0026360391639954407
0:	learn: 0.9647074	test: 0.9625819	best: 0.9625819 (0)	total: 817us	remaining: 817ms
500:	learn: 0.0044702	test: 0.0046456	best: 0.0046456 (500)	total: 346ms	remaining: 345ms
999:	learn: 0.0025553	test: 0.0027601	best: 0.0027601 (999)	total: 671ms	remaining: 0us

bestTest = 0.002760124255
bestIteration = 999

Fold 1. Score: 0.002760124255276898
Mean score: 0.0027
Std: 0.0001


In [15]:
get_results(cv=1, regime="global", y_true=test, y_pred=pred, ids=ids)

,y_true_1,y_pred_1,id_1
0,1997.0,1994.048102,0
1,1998.0,1994.405436,0
2,1999.0,1995.303323,0
3,2997.0,2994.048102,1
4,2998.0,2994.405436,1
5,2999.0,2995.303323,1
6,3997.0,3994.048102,2
7,3998.0,3994.405436,2
8,3999.0,3995.303323,2
9,4997.0,4994.048102,3


## Working with raw time series' granularity

Time series come in different granularities, from hourly and daily time series to more complex ones such as the end of each quarter.

If the rows do not contain segments that are too short (that are shorter than history + horizon), then `tsururu` will try to extract the row granularity on its own. We currently support the following types:

- Yearly (and YearlyEnd)
- Quarterly (and Quarterly)
- Monthly (and MonthlyEnd)
- Weekly
- Daily
- Hourly
- Minlutely
- Secondly
- Microsecondly

There is also support for compound granularities (10 days, 15 minutes, 32 seconds, etc.). The correctness of the selected granularity can be checked from the output after the `Dataset` class has been created.

However, there are tricky situations (e.g. 28 days) where the monthly granularity may be guessed incorrectly. Therefore, it is possible to set your own granularity using the `pd.DateOffset` class or related classes from `pandas.tseries.offsets`, which must be fed as `delta` parameter into the `Dataset` class. Then the time column will be processed according to the user's settings.

Consider a time series where each point is exactly __28 daily points away__ from each other

In [16]:
df_path_2 = "datasets/global/simulated_data_to_check_28D.csv"

# Configure the features settings
dataset_params_2 = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [17]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    print_freq_period_info=True,
)

freq: Month; period: 1.0


We see that the frequency of the series is incorrectly defined as monthly. Let's try to pass the `delta` parameter.

In [18]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    delta=pd.DateOffset(days=28),
    print_freq_period_info=True,
)

Custom OffSet: <DateOffset: days=28>


Now it's all detected correctly.